In [61]:
import pandas as pd
import numpy as np
import Memory as mm
import time
from scipy.stats import chi2_contingency

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [42]:
jiben = pd.read_csv('refer/jiben_vars_new_20200424.csv',skiprows=list(range(779441,779509)),
                    encoding="ISO-8859-1",error_bad_lines=False,low_memory=False)
jiben = mm.reduce_mem_usage(jiben)
strange_bad = jiben['bad'].value_counts().index[-1]
jiben['bad'].replace({strange_bad:'N'},inplace=True)
jiben['bad'].fillna('N',inplace=True)

Reduce Memory Usage Function Reports:
Memory usage of dataframe is 1259.57 MB
Memory usage after optimization is: 869.85 MB
Decreased by 30.9%
 


In [43]:
jiben.shape

(1634600, 101)

In [9]:
def Exploration(df,almost_empty=0.9):

    start = time.time()
    
    n,m = df.shape[0],df.shape[1] 
    dtypes = pd.DataFrame(df.dtypes)
    
    # Missing Values, Nunique, Data Type
    dtypes.columns = ['data types']
    null = pd.DataFrame(df.isnull().sum())
    null.columns = ['none cnt']
    null['none percentage'] = (null['none cnt']/n).apply(lambda x: "{0:.2f}%".format(x * 100))
    null['empty'] = 1*(null['none cnt']==n)
    null['>='+str(int(almost_empty*100))+'% empty']=1*(null['none cnt']/n>=almost_empty)
    
    nunique = pd.DataFrame(df.nunique())
    nunique.columns = ['n_unique']
    
    exploration = pd.concat([dtypes,null,nunique],axis=1)
    
    
    
    end = time.time()
    
    print('function finished one time, time cost {}seconds'.format(np.round(end-start,2)))
    
#   return [exploration,total_dup_cnt,unique_dup_cnt]
    return exploration

In [10]:
jiben_exp = Exploration(jiben)

function finished one time, time cost 14.84seconds


In [11]:
jiben_exp.to_csv('basic_info.csv',index=False)

In [44]:
vars_list = ['var_jb_86','var_jb_9','var_jb_39','var_jb_16','var_jb_2','var_jb_98','var_jb_82','var_jb_83',
             'var_jb_87','var_jb_78','var_jb_81','var_jb_76','var_jb_77','var_jb_93','var_jb_94']

In [45]:
shebao = pd.DataFrame(jiben['var_jb_86'].isnull()*1)
shebao.columns=['社保信息']

In [46]:
gongzi = jiben[['var_jb_9','var_jb_39']].isnull()*1
gongzi['工资信息'] = 1*((gongzi['var_jb_9']+gongzi['var_jb_39'])>0)

In [47]:
jinjian = pd.DataFrame(jiben['var_jb_16'].isnull()*1)
jinjian.columns = ['进件信息']

In [48]:
quyu = pd.DataFrame(jiben['var_jb_17'].isnull()*1)
quyu.columns = ['活动区域行为特征']

In [49]:
qudao = jiben[['var_jb_2','var_jb_98','var_jb_31']].isnull()*1
qudao['渠道服务信息'] = 1*((qudao['var_jb_2']+qudao['var_jb_98']+qudao['var_jb_31'])>0)

In [50]:
yuangong = jiben[['var_jb_82','var_jb_83','var_jb_87','var_jb_78','var_jb_81',
                  'var_jb_42','var_jb_40','var_jb_79','var_jb_80']].isnull()*1
yuangong['员工管理数据'] = 1*(yuangong.sum(axis=1)>0)

In [51]:
qiye = jiben[['var_jb_76','var_jb_77','var_jb_93','var_jb_94']].isnull()*1
qiye['企业外部评价数据'] = 1*(qiye.sum(axis=1)>0)

In [52]:
chanpin = pd.DataFrame(jiben['var_jb_15'].isnull()*1)
chanpin.columns = ['产品信息']

In [53]:
fengxian = jiben[['var_jb_30','var_jb_57']].isnull()*1
fengxian['风险特征'] = 1*(fengxian.sum(axis=1)>0)

In [55]:
id_bads = jiben[['id','bad']]

In [56]:
combined = pd.concat([id_bads,shebao,gongzi['工资信息'],jinjian,quyu,
                      qudao['渠道服务信息'],yuangong['员工管理数据'],qiye['企业外部评价数据'],
                      chanpin,fengxian['风险特征']],axis=1)

In [57]:
combined.head()

,id,bad,社保信息,工资信息,进件信息,活动区域行为特征,渠道服务信息,员工管理数据,企业外部评价数据,产品信息,风险特征
0,56a13c06ee2f2a0db827d0d9450213d3,G,0,0,0,0,0,1,1,0,1
1,c6342448d324d0e99e0d89175a2da335,G,0,1,0,0,0,1,1,0,1
2,82e5383d3b794f2555dadb7e37e9108e,G,0,1,0,0,0,1,1,0,1
3,a5e91382fc770d3bf1c3845d113d85a5,G,0,1,0,0,0,1,1,0,1
4,57e97bcbc4090d25d1c783fcb071b6df,G,0,1,0,0,0,1,1,0,1


In [58]:
combined.to_csv('outside_data_my_part_0429.csv',index=False)